# Spark DataFrame basic operations
### Dr. Tirthajyoti Sarkar, Fremont, CA, July 2020 (updated)
In this notebook, we go through basic operations that can be performed with a Spark DataFrame object. We will use a .CSV file of stock prices to illustrate the various useful methods that are associated with a Spark DataFrame.

---

In [1]:
from pyspark.sql import SparkSession

### A `SparkSession` app, reading CSV, and the Schema

In [2]:
spark1 = SparkSession.builder.appName('Ops').getOrCreate()

Read the Apple stock CSV file. Note we have an option of inferring the schema for CSV. We also have the option to set `header` to `True`.

In [3]:
df = spark1.read.csv('Data/stock.csv',inferSchema=True,header=True)

Print the schema

In [4]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



Show the DataFrame

In [5]:
df.show()

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-01-04 00:00:00|213.43| 214.5|212.38|214.01|123432400|    27.73|
|2010-01-05 00:00:00| 214.6|215.59|213.25|214.38|150476200|    27.77|
|2010-01-06 00:00:00|214.38|215.23|210.75|210.97|138040000|    27.33|
|2010-01-07 00:00:00|211.75| 212.0|209.05|210.58|119282800|    27.28|
|2010-01-08 00:00:00| 210.3| 212.0|209.06|211.98|111902700|    27.46|
|2010-01-11 00:00:00| 212.8| 213.0|208.45|210.11|115557400|    27.22|
|2010-01-12 00:00:00|209.19|209.77|206.42|207.72|148614900|    26.91|
|2010-01-13 00:00:00|207.87|210.93| 204.1|210.65|151473000|    27.29|
|2010-01-14 00:00:00|210.11|210.46|209.02|209.43|108223500|    27.13|
|2010-01-15 00:00:00|210.93| 211.6|205.87|205.93|148516900|    26.68|
|2010-01-19 00:00:00|208.33|215.19|207.24|215.04|182501900|    27.86|
|2010-01-20 00:00:00

---

### Show the columns

The `columns` attribute returns a simple Python list of column names!

In [6]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

---

### Spark DataFrames have separate `Column` and `Row` types 
Instead of generic Pandas series, Spark DataFrames have separate `Column` and `Row` types. This is ensure thet can handle special data files.

In [7]:
type(df['High'])

pyspark.sql.column.Column

In [8]:
type(df.head(2)[0])

pyspark.sql.types.Row

---

### The `select` method to select particular columns

We can use this method to actually select the DataFrame columns and see them. Note that we still have to use the `show` method to actually output the data.

In [9]:
df.select('High')

DataFrame[High: double]

In [10]:
df.select('High').show()

+------+
|  High|
+------+
| 214.5|
|215.59|
|215.23|
| 212.0|
| 212.0|
| 213.0|
|209.77|
|210.93|
|210.46|
| 211.6|
|215.19|
|215.55|
|213.31|
| 207.5|
| 204.7|
|213.71|
|210.58|
| 205.5|
| 202.2|
| 196.0|
+------+
only showing top 20 rows



In [11]:
df.select(['High','Close']).show()

+------+------+
|  High| Close|
+------+------+
| 214.5|214.01|
|215.59|214.38|
|215.23|210.97|
| 212.0|210.58|
| 212.0|211.98|
| 213.0|210.11|
|209.77|207.72|
|210.93|210.65|
|210.46|209.43|
| 211.6|205.93|
|215.19|215.04|
|215.55|211.73|
|213.31|208.07|
| 207.5|197.75|
| 204.7|203.07|
|213.71|205.94|
|210.58|207.88|
| 205.5|199.29|
| 202.2|192.06|
| 196.0|194.73|
+------+------+
only showing top 20 rows



---

### The `limit` method to take first few rows (without any collection)

Applying `limit()` to the DataFrame will result in a new Dataframe. This is a transformation and does not perform collecting the data. Other similar methods like `take` and `head` result in an Array of Rows i.e. a Scala Collection Object like `scala.collection.immutable.Array` (which is transformed to Python list while using the PySpark API).

In [12]:
df.limit(5)

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

In [13]:
df.limit(5).show()

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-01-04 00:00:00|213.43| 214.5|212.38|214.01|123432400|    27.73|
|2010-01-05 00:00:00| 214.6|215.59|213.25|214.38|150476200|    27.77|
|2010-01-06 00:00:00|214.38|215.23|210.75|210.97|138040000|    27.33|
|2010-01-07 00:00:00|211.75| 212.0|209.05|210.58|119282800|    27.28|
|2010-01-08 00:00:00| 210.3| 212.0|209.06|211.98|111902700|    27.46|
+-------------------+------+------+------+------+---------+---------+



---

### The `head()` and the `asDict()` methods

In [14]:
df.head(2)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.43, High=214.5, Low=212.38, Close=214.01, Volume=123432400, Adj Close=27.73),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.6, High=215.59, Low=213.25, Close=214.38, Volume=150476200, Adj Close=27.77)]

In [15]:
dict1=df.head(2)[0].asDict()

In [16]:
dict1

{'Date': datetime.datetime(2010, 1, 4, 0, 0),
 'Open': 213.43,
 'High': 214.5,
 'Low': 212.38,
 'Close': 214.01,
 'Volume': 123432400,
 'Adj Close': 27.73}

---

### The `count` method - number of rows

In [17]:
df.count()

1762

---

### Coutning rows with distinct values - `distinct` method

To demonstrate this method, we read in another small CSV file.

In [18]:
df2 = spark1.read.csv('Data/sales_info.csv',inferSchema=True,header=True)
df2.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [19]:
df2.select('Company').distinct().show()

+-------+
|Company|
+-------+
|   APPL|
|   GOOG|
|     FB|
|   MSFT|
+-------+



In [20]:
df2.select('Company').distinct().count()

4

---

### Random sampling using the `sample` method

In [21]:
df.sample(withReplacement=False,
          fraction = 0.005,
          seed = 101).show()

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-03-19 00:00:00|224.79|225.24|221.23|222.25|139861400|    28.79|
|2010-04-09 00:00:00|241.43|241.89|240.46|241.79| 83545700|    31.33|
|2010-06-07 00:00:00|258.29|259.15|250.55|250.94|221735500|    32.51|
|2011-07-19 00:00:00| 378.0|378.65|373.32|376.85|204786400|    48.82|
|2012-07-24 00:00:00|607.38|609.68|598.51|600.92|141283100|    77.85|
|2013-09-04 00:00:00|499.56|502.24|496.28|498.69| 86258200|    66.43|
|2013-11-12 00:00:00|517.67|523.92| 517.0|520.01| 51069200|    69.68|
|2013-11-19 00:00:00|519.03|523.38|517.97|519.55| 52234700|    69.62|
|2014-02-06 00:00:00|510.06| 513.5|507.81|512.51| 64441300|    69.08|
|2014-05-09 00:00:00|584.54|586.25|580.33|585.54| 72899400|    79.37|
|2014-08-11 00:00:00| 95.27| 96.08| 94.84| 95.99| 36585000|    91.53|
|2015-01-16 00:00:00

---

### Inserting new column using `withColumn` method

We can insert a new column into the DataFrame using the `withColumn()` method. It is mostly used when we are doing some transformartion of the data using one or more columns. Following is a simple example where we are calculating the range of the stock price (difference between the `High` and `Low` columns).

Also note how we are **chaining the methods** - a `withColumn`, followed by a `limit`, followed by a `select` to show only the columns impacted (in a certain order), and finally followed by a `show` to see the final result of only first five rows.

In [22]:
df.withColumn('Range',df['High']-df['Low']).limit(5).select(['High','Low','Range']).show()

+------+------+------------------+
|  High|   Low|             Range|
+------+------+------------------+
| 214.5|212.38|2.1200000000000045|
|215.59|213.25|2.3400000000000034|
|215.23|210.75|  4.47999999999999|
| 212.0|209.05|2.9499999999999886|
| 212.0|209.06|2.9399999999999977|
+------+------+------------------+



This is **not an _in-place_ operation** i.e. the original DataFrame remians unchanged.

---

### Renaming a column using `withColumnRenamed` method

In [23]:
df3 = df.withColumn('Range',df['High']-df['Low']).limit(5).select(['High','Low','Range']) 

In [24]:
df3.show()

+------+------+------------------+
|  High|   Low|             Range|
+------+------+------------------+
| 214.5|212.38|2.1200000000000045|
|215.59|213.25|2.3400000000000034|
|215.23|210.75|  4.47999999999999|
| 212.0|209.05|2.9499999999999886|
| 212.0|209.06|2.9399999999999977|
+------+------+------------------+



In [25]:
# Just pass the old and new column names
df3 = df3.withColumnRenamed('Range','Min-to-Max')
df3.show()

+------+------+------------------+
|  High|   Low|        Min-to-Max|
+------+------+------------------+
| 214.5|212.38|2.1200000000000045|
|215.59|213.25|2.3400000000000034|
|215.23|210.75|  4.47999999999999|
| 212.0|209.05|2.9499999999999886|
| 212.0|209.06|2.9399999999999977|
+------+------+------------------+



---

### Filtering Operations - using either `filter()` or `where()` method

Filtering can be done with **SQL-like syntax** or **Pythonic** way. We show both examples.

Pass on a SQL syntax to the `filter()` method

In [26]:
df.filter("Close < 500").show(5)

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-01-04 00:00:00|213.43| 214.5|212.38|214.01|123432400|    27.73|
|2010-01-05 00:00:00| 214.6|215.59|213.25|214.38|150476200|    27.77|
|2010-01-06 00:00:00|214.38|215.23|210.75|210.97|138040000|    27.33|
|2010-01-07 00:00:00|211.75| 212.0|209.05|210.58|119282800|    27.28|
|2010-01-08 00:00:00| 210.3| 212.0|209.06|211.98|111902700|    27.46|
+-------------------+------+------+------+------+---------+---------+
only showing top 5 rows



We can chain methods to see only desired columns

In [27]:
df.filter("Close < 500 AND Open > 500").select(['Date','Open','Close']).show()

+-------------------+------+------+
|               Date|  Open| Close|
+-------------------+------+------+
|2012-02-15 00:00:00|514.26|497.67|
|2013-09-05 00:00:00|500.25|495.27|
|2013-09-10 00:00:00| 506.2|494.64|
|2014-01-30 00:00:00|502.54|499.78|
+-------------------+------+------+



In [28]:
df.where("Open < 500 AND (Open-Close)> 10").select(['Date','Open','Close']).show(5)

+-------------------+------+------+
|               Date|  Open| Close|
+-------------------+------+------+
|2010-07-21 00:00:00|265.09|254.24|
|2011-03-16 00:00:00| 342.0|330.01|
|2011-08-04 00:00:00|389.41|377.37|
|2011-09-29 00:00:00|401.92|390.57|
|2011-11-10 00:00:00|397.03|385.22|
+-------------------+------+------+
only showing top 5 rows



---

### Now we use DataFrame syntax to achieve the same output

In [29]:
df.filter(df['Close']<500).show(5)

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-01-04 00:00:00|213.43| 214.5|212.38|214.01|123432400|    27.73|
|2010-01-05 00:00:00| 214.6|215.59|213.25|214.38|150476200|    27.77|
|2010-01-06 00:00:00|214.38|215.23|210.75|210.97|138040000|    27.33|
|2010-01-07 00:00:00|211.75| 212.0|209.05|210.58|119282800|    27.28|
|2010-01-08 00:00:00| 210.3| 212.0|209.06|211.98|111902700|    27.46|
+-------------------+------+------+------+------+---------+---------+
only showing top 5 rows



**If we need to chain multiple conditions together, use `&` for AND and `|` for OR and clearly separate the conditions by putting them inside parantheses.**

In [30]:
df.filter((df['Close']<500) & (df['Open']>500)).select(['Date','Open','Close']).show(5)

+-------------------+------+------+
|               Date|  Open| Close|
+-------------------+------+------+
|2012-02-15 00:00:00|514.26|497.67|
|2013-09-05 00:00:00|500.25|495.27|
|2013-09-10 00:00:00| 506.2|494.64|
|2014-01-30 00:00:00|502.54|499.78|
+-------------------+------+------+



In [31]:
df.filter((df['Close']<500) & (df['Open']-df['Close']>10)).select(['Date','Open','Close']).show(5)

+-------------------+------+------+
|               Date|  Open| Close|
+-------------------+------+------+
|2010-07-21 00:00:00|265.09|254.24|
|2011-03-16 00:00:00| 342.0|330.01|
|2011-08-04 00:00:00|389.41|377.37|
|2011-09-29 00:00:00|401.92|390.57|
|2011-11-10 00:00:00|397.03|385.22|
+-------------------+------+------+
only showing top 5 rows



We can use `==` to compare with an exact value for comparison and `~` for NOT operator

In [32]:
df.filter(df['Low']==197.16).show()

+-------------------+------+-----+------+------+---------+---------+
|               Date|  Open| High|   Low| Close|   Volume|Adj Close|
+-------------------+------+-----+------+------+---------+---------+
|2010-01-22 00:00:00|206.78|207.5|197.16|197.75|220441900|    25.62|
+-------------------+------+-----+------+------+---------+---------+



In [33]:
df.filter((df['Open']<200) & ~(df['Close']>200)).show(5)

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-02-01 00:00:00|192.37| 196.0| 191.3|194.73|187469100|    25.23|
|2010-02-02 00:00:00|195.91|196.32|193.38|195.86|174585600|    25.38|
|2010-02-03 00:00:00|195.17| 200.2|194.42|199.23|153832000|    25.81|
|2010-02-04 00:00:00|196.73|198.37|191.57|192.05|189413000|    24.88|
|2010-02-05 00:00:00|192.63| 196.0|190.85|195.46|212576700|    25.32|
+-------------------+------+------+------+------+---------+---------+
only showing top 5 rows



---

### Use the `collect` method instead of `show`, to collect the actual data

In [34]:
low_data = df.filter(df['Low']==197.16).collect()

In [35]:
low_data

[Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78, High=207.5, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.62)]

#### It is still a list. So, grab the 0-index element as a Row object and convert it to a dictionary using `asDict` method

In [36]:
dt = low_data[0]

In [37]:
dt.asDict()

{'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'Open': 206.78,
 'High': 207.5,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.62}

Now, you can do whatever processing you want to do with the dictionary object!

---

### Sorting DataFrame - `sort` and `orderBy` methods

We create a smaller DataFrame from the original one.

In [38]:
df4 = df.select('High','Low').limit(10)
df4.show()

+------+------+
|  High|   Low|
+------+------+
| 214.5|212.38|
|215.59|213.25|
|215.23|210.75|
| 212.0|209.05|
| 212.0|209.06|
| 213.0|208.45|
|209.77|206.42|
|210.93| 204.1|
|210.46|209.02|
| 211.6|205.87|
+------+------+



In [39]:
df4.sort("High").show()

+------+------+
|  High|   Low|
+------+------+
|209.77|206.42|
|210.46|209.02|
|210.93| 204.1|
| 211.6|205.87|
| 212.0|209.05|
| 212.0|209.06|
| 213.0|208.45|
| 214.5|212.38|
|215.23|210.75|
|215.59|213.25|
+------+------+



By default, it sorts by ascending order. You can change it by passing `ascending=False`

In [40]:
df4.sort("High", ascending=False).show()

+------+------+
|  High|   Low|
+------+------+
|215.59|213.25|
|215.23|210.75|
| 214.5|212.38|
| 213.0|208.45|
| 212.0|209.05|
| 212.0|209.06|
| 211.6|205.87|
|210.93| 204.1|
|210.46|209.02|
|209.77|206.42|
+------+------+



You can also use the **`desc` function** imported from the `pyspark.sql.functions` module

In [41]:
from pyspark.sql.functions import desc
df4.sort(desc("High")).show()

+------+------+
|  High|   Low|
+------+------+
|215.59|213.25|
|215.23|210.75|
| 214.5|212.38|
| 213.0|208.45|
| 212.0|209.05|
| 212.0|209.06|
| 211.6|205.87|
|210.93| 204.1|
|210.46|209.02|
|209.77|206.42|
+------+------+



You can pass a list to the `ascending` parameter to create custom sorting order on various columns. In the following example, we are sorting in descending order for the `Company` column, while doing an ascending sort on the `Sales` column. We are using the `orderBy` method for this one.

In [42]:
df2.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [43]:
df2.orderBy(["Company","Sales"],ascending=[0,1]).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|   MSFT|   Tina|600.0|
|   GOOG|Charlie|120.0|
|   GOOG|    Sam|200.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|     FB|   Carl|870.0|
|   APPL|  Linda|130.0|
|   APPL|   John|250.0|
|   APPL|  Chris|350.0|
|   APPL|   Mike|750.0|
+-------+-------+-----+



---

### Using functions from the `pyspark.sql.functions` module

We can utilize various utility functions from the `pyspark.sql.functions` module to do transformation on the data and extract insights, look for patterns, build statistical models, etc.

We will just show a couple of examples here, but you are encouraged to explore others. Here is the detailed documentation link: 

http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [44]:
from pyspark.sql.functions import corr, dayofweek, month

Suppose, we want to check ***if there is any correlation between the trade volumes and the range (Max - Min) i.e. intra-day volatility of the stock***.

Here is how we can do that computation. Note the use of two new methods - `agg` which is nothing but an aggregration over the entire DataFrame without forming any specific groups and the `alias` method which is just a way to specify a descriptive name for the column resulting from the application of the `corr` function. We need the `agg` here because we want to aggregrate and throw all the data of the DataFrame at the `corr` function to calculate the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between the `Volume` and `Range` data columns.

In [45]:
df.withColumn('Range',df['High']-df['Low']).agg(corr("Volume","Range").alias('Pearson-Corr')).collect()

[Row(Pearson-Corr=0.6980419831842759)]

So, we do have a fairly strong positive correlation between the `Volume` and `Range` data.

Here is another fun observation! If you stare at the code above a bit longer, you will notice that **we did not produce a standalone `Range` column at all**. We computed it in an ephemeral manner - just enough to calculate the correlation coefficient. The original DataFrame remains unchanged while we got our answer about the positive correlation!

In [46]:
df.show(5)

+-------------------+------+------+------+------+---------+---------+
|               Date|  Open|  High|   Low| Close|   Volume|Adj Close|
+-------------------+------+------+------+------+---------+---------+
|2010-01-04 00:00:00|213.43| 214.5|212.38|214.01|123432400|    27.73|
|2010-01-05 00:00:00| 214.6|215.59|213.25|214.38|150476200|    27.77|
|2010-01-06 00:00:00|214.38|215.23|210.75|210.97|138040000|    27.33|
|2010-01-07 00:00:00|211.75| 212.0|209.05|210.58|119282800|    27.28|
|2010-01-08 00:00:00| 210.3| 212.0|209.06|211.98|111902700|    27.46|
+-------------------+------+------+------+------+---------+---------+
only showing top 5 rows



Next, we apply the `dayofweek` and `month` functions on the `Date` column to produce new columns `Day-of-Week` and `Month`. Note how we chained two `withColumn` methods together for this transformation.

In [47]:
df5=df.withColumn('Day-of-Week',dayofweek('Date')).\
withColumn('Month',month('Date')).\
select(['Day-of-Week','Month','Open','Close'])

df5.show(5)

+-----------+-----+------+------+
|Day-of-Week|Month|  Open| Close|
+-----------+-----+------+------+
|          2|    1|213.43|214.01|
|          3|    1| 214.6|214.38|
|          4|    1|214.38|210.97|
|          5|    1|211.75|210.58|
|          6|    1| 210.3|211.98|
+-----------+-----+------+------+
only showing top 5 rows



Now, we have not shown the `groupby()` and aggregration (like average or `mean`) methods yet but if you have a fair understanding of what a `groupby` (followed by an aggregration like `mean`) does, then you can appreciate the following example where we look at the average `Volume` grouped by `Day-of-Week` and `Month` and try to gauge ***which day of the week or month have the highest and lowest average trade volumes***.

The example below is certainly a complex one but you can think this one as an illustration of using some of the most important DataFrame methods we have examined in this Notebook (and some future ones we will look at), in a combined fashion.

- `withColumn`
- `select`
- `orderBy`
- `groupBy`
- `mean` (aggregration)

In [48]:
df5=df.withColumn('Day-of-Week',dayofweek('Date')).\
withColumn('Month',month('Date')).\
select(['Day-of-Week','Month','Volume']).\
groupby(['Day-of-Week','Month']).\
mean().\
select(['Day-of-Week','Month','avg(Volume)']).\
orderBy(['Day-of-Week','Month'])

df5.show(60)

+-----------+-----+--------------------+
|Day-of-Week|Month|         avg(Volume)|
+-----------+-----+--------------------+
|          2|    1|1.0967633636363636E8|
|          2|    2| 8.957742272727273E7|
|          2|    3| 9.421765161290322E7|
|          2|    4|1.0364189333333333E8|
|          2|    5| 9.646600416666667E7|
|          2|    6| 8.898051666666667E7|
|          2|    7| 8.163605555555555E7|
|          2|    8|      8.5287746875E7|
|          2|    9|  9.82580956521739E7|
|          2|   10| 8.972772333333333E7|
|          2|   11|          7.374762E7|
|          2|   12| 7.757132068965517E7|
|          3|    1|1.3081937586206897E8|
|          3|    2|1.0092741428571428E8|
|          3|    3|      1.0023139375E8|
|          3|    4|1.0715501333333333E8|
|          3|    5|1.0042301290322581E8|
|          3|    6| 8.865190333333333E7|
|          3|    7| 9.382107096774194E7|
|          3|    8| 9.748764193548387E7|
|          3|    9|          9.932478E7|
|          3|   